# Quickstart

Get up and running with MolPy in just a few minutes!  
This page gives you a **fast tour of the core data structures and features**:

- `Frame` – tabular numerical data (coordinates, charges, etc.)
- `Atomistic` – graph/topology representation (atoms, bonds, reactions)
- `Box` – simulation cell and periodic boundary conditions
- `io` – read/write common file formats like PDB/LAMMPS

Once you've run through these short examples, you can dive into the detailed tutorials.

---

## 1. Imports



In [1]:
import numpy as np
import molpy as mp


---

## 2. Frame: tabular view of a structure

`Frame` is a hierarchical container of `Block` objects (like `"atoms"`, `"bonds"`), plus a `metadata` dict.
It's ideal for storing per-atom/per-bond arrays in a NumPy‑friendly way.



In [2]:
# Create a simple system with three carbon atoms in a box
box = mp.Box.cubic(10.0)  # 10 × 10 × 10 Å

frame = mp.Frame()
atoms = mp.Block()
atoms["x"] = [0.0, 1.0, 2.0]
atoms["y"] = [0.0, 0.0, 0.0]
atoms["z"] = [0.0, 0.0, 0.0]
atoms["element"] = ["C", "C", "C"]
atoms["vector"] = np.random.randn(3, 3)  # vector per atom

# Add atoms block to frame
frame["atoms"] = atoms

print(f"block: {atoms}")

# Store box in metadata
frame.metadata["box"] = box

print("Number of atoms:", frame["atoms"].nrows)
print("Box lengths:", box.lengths)

# Access combined coordinates as a single array
atoms = frame["atoms"]
xyz = atoms[["x", "y", "z"]]
print("XYZ shape:", xyz.shape)

block: Block(x: shape=(3,), y: shape=(3,), z: shape=(3,), element: shape=(3,), vector: shape=(3, 3))
Number of atoms: 3
Box lengths: [10. 10. 10.]
XYZ shape: (3, 3)


For a deeper dive into Frames and Blocks, see the [**Frame & Block**](../../tutorials/frame-block) tutorial

---

## 3. Atomistic: topology and chemistry

`Atomistic` represents a molecular system as atoms and bonds.  
It is the core structure used by the **reacter**, **typifier**, and other chemistry‑aware modules.

In [3]:
from molpy import Atom, Atomistic, Bond

# Create a small C–H2 fragment
atomistic = Atomistic()
c = Atom(symbol="C")
h1 = Atom(symbol="H")
h2 = Atom(symbol="H")

atomistic.add_atom(c)
atomistic.add_atom(h1)
atomistic.add_atom(h2)
atomistic.add_bond(Bond(c, h1))
atomistic.add_bond(Bond(c, h2))

print("Atomistic:", atomistic)  # concise summary
print("Number of atoms:", len(list(atomistic.atoms)))
print("Number of bonds:", len(list(atomistic.bonds)))

Atomistic: <Atomistic, 3 atoms (C:1 H:2), 2 bonds>
Number of atoms: 3
Number of bonds: 2



From here you can:

- Build monomers and polymers with wrappers like `Monomer`
- Run polymerization or reaction workflows with `molpy.reacter`

See the [**molecular modeling / reaction**](/docs/tutorials/reaction-modeling)

---

## 4. Box: simulation cell and PBC

`Box` defines your simulation cell and periodic boundary conditions (PBC):



In [4]:
box = mp.Box.cubic(20.0, pbc=[True, True, True])

print("Box style:", box.style)
print("Lengths:", box.lengths)
print("PBC:", box.pbc)
print("Volume:", box.volume)

Box style: Style.ORTHOGONAL
Lengths: [20. 20. 20.]
PBC: [ True  True  True]
Volume: 8000.0



For more on box styles (orthogonal, triclinic, centered boxes), see:
[Box tutorial](/docs/tutorials/box).

---

## 5. File I/O: reading and writing PDB

The `molpy.io` module provides readers and writers for common formats.
You can write any `Frame` that has an `"atoms"` block with coordinates:

In [5]:
from molpy.io import read_pdb, write_pdb
import numpy as np

# Ensure we have xyz coordinates for PDB writing
if "xyz" not in frame["atoms"].keys():
    # Create xyz from x, y, z
    x = frame["atoms"]["x"]
    y = frame["atoms"]["y"]
    z = frame["atoms"]["z"]
    frame["atoms"]["xyz"] = np.column_stack([x, y, z])

write_pdb("output.pdb", frame)
loaded_frame = read_pdb("output.pdb")

print("Loaded frame atoms:", loaded_frame["atoms"].nrows)

Loaded frame atoms: 3



Other formats (LAMMPS data/trajectory, GRO, XYZ, etc.) are covered in the **File I/O** user guide.

---

## 6. Where to go next

- [Architecture](/docs/getting-started/core-concepts.md): high‑level architecture and data model  
- [API Reference](/docs/api/index.md): full Python API  